# A Guided Tour of Ray Core: Remote Functions

[*Remote Functions*](https://docs.ray.io/en/latest/walkthrough.html#remote-functions-tasks)
involve using a `@ray.remote` decorator on a function. 

This implements a [*task parallelism*](https://patterns.eecs.berkeley.edu/?page_id=208) pattern, with properties: *data independence*, *stateless*

But first the concept of tasks and ownership.

Most of the system metadata is managed according to a decentralized concept called ownership: Each worker process manages and owns the tasks that it submits and the `ObjectRef`s returned by those tasks. The owner is responsible for ensuring execution of the task and facilitating the resolution of an `ObjectRef` to its underlying value. Similarly, a worker owns any objects that it created through a `ray.put` call.

<img src="images/task_ownership.png" height=350, width=650>


---

### Lifetime of a Ray Task

The owner is responsible for ensuring execution of a submitted task and facilitating the resolution of the returned `ObjectRef` to its underlying value.
The process that submits a task is considered to be the owner of the result and is responsible for acquiring resources from the raylet to execute the task. Here, the driver owns the result of `A`, and `Worker 1` owns the result of `B`.

<img src="images/task_life.png" height=350, width=650>

### Launch a Ray locally, a single head node

First, let's start Ray… 

This will start Ray on the local host, with headnode and workers for each core or CPU available.
You can check the resources being used.

In [19]:
import logging
import ray

ray.init(
    ignore_reinit_error=True,              # Don't print error messages if a Ray instance is already running. Attach to it
    logging_level=logging.ERROR,           
)
ray.cluster_resources()                    # get the cluster resources

{'CPU': 12.0,
 'memory': 13590429696.0,
 'node:127.0.0.1': 1.0,
 'object_store_memory': 6795214848.0}

Ray Dashboard accessible at URI: [http://127.0.0.1:8265](http://127.0.0.1:8265)

## Remote Functions example

The following is just a regular Python function...

In [20]:
def my_function ():
    return 42

When called, it simply returns an integer:

In [21]:
my_function()

42

If you were to iterate through a sequence of calls to a function such as that, these calls would be performed *sequentially*.

However, by adding the `@ray.remote` decorator, a regular Python function becomes a Ray remote function:

In [22]:
@ray.remote
def my_function ():
    return 42

To invoke this remote function, use the `remote` method. This will immediately return an object ref (a *future* in Python) and then create a task that will be executed on a worker process.

In [23]:
%%time

obj_ref = my_function.remote()
obj_ref

CPU times: user 1.42 ms, sys: 1.11 ms, total: 2.53 ms
Wall time: 1.79 ms


ObjectRef(bbde8638d39a1245ffffffffffffffffffffffff0100000001000000)

The result can be retrieved with `ray.get`, which is a blocking call if the task is still not finished

In [24]:
%%time

ray.get(obj_ref)

CPU times: user 205 µs, sys: 99 µs, total: 304 µs
Wall time: 286 µs


42

Invocations of Ray *remote functions* happen in parallel, and all computation gets performed in the background, driven by Ray's internal event loop. Remote calls return immediately, with a Python Future Object reference.

To illustrate this, first let's define a relatively "slow" function, by adding a 10 second delay...

In [25]:
import time
import random

@ray.remote
def slow_function ():
  time.sleep(10)
  return random.randint(0, 9)

Now we'll iterate through multiple calls, showing that this does not block:

In [26]:
%%time

futures_list = []

for i in range(4):
    future = slow_function.remote()
    futures_list.append(future)
    print(i)
print(futures_list)

0
1
2
3
[ObjectRef(44ed5e1383be6308ffffffffffffffffffffffff0100000001000000), ObjectRef(d56d800cbde6ca14ffffffffffffffffffffffff0100000001000000), ObjectRef(38db0ab51c6b6cfbffffffffffffffffffffffff0100000001000000), ObjectRef(4e2ab276f14c37c2ffffffffffffffffffffffff0100000001000000)]
CPU times: user 2.31 ms, sys: 1.84 ms, total: 4.15 ms
Wall time: 2.31 ms


In [27]:
%%time

for future in futures_list:
    print(ray.get(future))

5
9
9
5
CPU times: user 288 ms, sys: 184 ms, total: 472 ms
Wall time: 8.77 s


In [ ]:
%%time

futures_list = []

for i in range(4):
    future = slow_function.remote()
    futures_list.append(future)
print(futures_list)

In [ ]:
%%time

for future in futures_list:
    print(ray.get(future))

Note the difference between CPU times and wall clock?

## Another way to do this is using list comprehension

In [30]:
%%time
futures_list = [slow_function.remote() for _ in range(4)]
futures_list

CPU times: user 1.14 ms, sys: 524 µs, total: 1.66 ms
Wall time: 838 µs


[ObjectRef(d5a75db31f99bd73ffffffffffffffffffffffff0100000001000000),
 ObjectRef(d68fec326c8433c9ffffffffffffffffffffffff0100000001000000),
 ObjectRef(8e088f779f48acd6ffffffffffffffffffffffff0100000001000000),
 ObjectRef(65a1a3aaa614cba3ffffffffffffffffffffffff0100000001000000)]

In [31]:
values = [ray.get(future) for future in futures_list]
values

[7, 6, 2, 1]

In [ ]:
%%time

futures_list = [slow_function.remote() for _ in range(4)]
futures_list

In [ ]:
%%time

values_list = [ray.get(future) for future in futures_list]
values_list

Note the difference between CPU times and wall clock in comprehensions? 
Comprehensions seems faster.

Finally, shutdown Ray

In [34]:
ray.shutdown()

---
## References

[*Patterns for Parallel Programming*](https://www.goodreads.com/book/show/85053.Patterns_for_Parallel_Programming)  
Timothy G. Mattson, Beverly A. Sanders, Berna L. Massingill  
Addison-Wesley (2004)

[Ray Core Walkthrough](https://docs.ray.io/en/latest/walkthrough.html)
Ray Documentation and Gettting started materials

[Ray Architecture Reference](https://docs.google.com/document/d/1lAy0Owi-vPz2jEqBSaHNQcy2IBSDEHyXNOQZlGuj93c/preview#)
Ray 1.x Architecture Technical Paper

[Ray Internals: A peek at ray,get](https://www.youtube.com/watch?v=a1kNnQu6vGw)

[Ray Internals: Object management with Ownership Model](https://www.anyscale.com/events/2021/06/22/ray-internals-object-management-with-the-ownership-model)